
## This version of build_vector_db.py and asupport_chat.py just have each section of the code clearly deliniated for a class demo


In [1]:
# ==============================================================
# build_vector_db.py - FINAL WORKING VERSION
# PIPELINE: 0 → Ingest | 1 → Chunk | 2 → Embed | 3 → Index
# ==============================================================

import os
from pathlib import Path
from typing import List

from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    Distance,
    VectorParams,
    HnswConfigDiff,
    PointStruct,
)

# CONFIG
DATA_DIR = "support_docs"
COLLECTION_NAME = "support_docs"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
QDRANT_URL = "http://localhost:6333"


# ==============================================================
# STEP 0: INGEST RAW DOCUMENTS
# Load PDFs, .txt, .md from support_docs/
# ==============================================================
def load_documents() -> List[Document]:
    # ── 1. Print header for clarity in console
    print("\n" + "="*60)
    print("STEP 0: INGEST RAW DOCUMENTS")
    print("="*60)

    # ── 2. Define where to look for files
    data_dir = Path(DATA_DIR)  # DATA_DIR = "support_docs"

    # ── 3. Safety check: folder must exist
    if not data_dir.exists():
        raise FileNotFoundError(f"Create '{DATA_DIR}' folder with your files.")

    # ── 4. Map file extensions → loader classes
    loaders = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".md":  UnstructuredMarkdownLoader
    }

    # ── 5. List to store all loaded document objects
    docs = []

    # ── 6. Start loading
    print("Loading documents...")

    # ── 7. Recursively search ALL files in support_docs/ and subfolders
    for file_path in data_dir.rglob("*"):
        ext = file_path.suffix.lower()  # e.g., ".pdf"

        # ── 8. Only process supported file types
        if ext in loaders:
            print(f"  → {file_path.name}")  # Show filename

            # ── 9. Load the file using the correct loader
            loader = loaders[ext](str(file_path))  # Create loader instance
            for doc in loader.load():              # .load() returns list of Document
                # ── 10. Add useful metadata
                doc.metadata.update({
                    "source": file_path.name,         # e.g., "Supportco Manual.pdf"
                    "category": file_path.parent.name # e.g., "onboarding", "troubleshooting"
                })
                docs.append(doc)  # Save each page/section as a Document

    # ── 11. Summary
    print(f"Loaded {len(docs)} sections.")
    return docs  # Return list of Document objects


# ==============================================================
# STEP 1: CHUNKING (content-aware)
# Split long docs into 500-char chunks with 50 overlap
# ==============================================================
def chunk_documents(docs: List[Document]) -> List[Document]:
    # ── 1. Print header
    print("\n" + "="*60)
    print("STEP 1: CHUNKING")
    print("="*60)

    # ── 2. Create a text splitter with settings
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,      # e.g., 500 characters
        chunk_overlap=CHUNK_OVERLAP # e.g., 50 characters
    )

    # ── 3. Split all documents into smaller chunks
    chunks = splitter.split_documents(docs)

    # ── 4. Summary
    print(f"Created {len(chunks)} chunks.")
    return chunks  # List of smaller Document objects


# ==============================================================
# STEP 2: EMBEDDING
# Convert each chunk into a 384-dim vector using all-MiniLM-L6-v2
# ==============================================================
# ==============================================================
def embed_chunks(chunks: List[Document]) -> List[dict]:
    # ── 1. Print header
    print("\n" + "="*60)
    print("STEP 2: EMBEDDING")
    print("="*60)

    # ── 2. Load the embedding model (once)
    model = SentenceTransformer(EMBEDDING_MODEL)  # e.g., "all-MiniLM-L6-v2"

    # ── 3. List to store results
    embedded = []

    # ── 4. Start embedding
    print("Generating embeddings...")

    # ── 5. Loop through each chunk
    for i, c in enumerate(chunks):
        # ── 6. Convert text → 384-dimensional vector
        vec = model.encode(c.page_content).tolist()

        # ── 7. Save: ID, vector, and original text + source
        embedded.append({
            "id": i,                                      # Unique ID
            "vector": vec,                                # [0.1, -0.3, ..., 0.7] (384 numbers)
            "payload": {
                "text": c.page_content,                   # Original chunk
                "source": c.metadata.get("source")        # e.g., "Supportco Manual.pdf"
            }
        })

        # ── 8. Progress update every 100 chunks
        if i % 100 == 0 and i > 0:
            print(f"  → Embedded {i}/{len(chunks)} chunks")

    # ── 9. Final summary
    print(f"Embedded {len(embedded)} vectors.")
    return embedded  # List of dicts ready for Qdrant


# ==============================================================
# STEP 3: INDEXING & STORAGE IN QDRANT
# Create collection + upsert all vectors
# ==============================================================
def build_qdrant_index(points: List[dict]):
    # ── 1. Print header
    print("\n" + "="*60)
    print("STEP 3: INDEXING & STORAGE IN QDRANT")
    print("="*60)

    # ── 2. Connect to Qdrant (running in Docker)
    client = QdrantClient(url=QDRANT_URL)  # QDRANT_URL = "http://localhost:6333"

    # ── 3. Delete old collection if it exists
    if client.collection_exists(COLLECTION_NAME):
        client.delete_collection(COLLECTION_NAME)
        print(f"Deleted existing collection: {COLLECTION_NAME}")

    # ── 4. Create a new collection
    client.create_collection(
        collection_name=COLLECTION_NAME,  # e.g., "support_docs"
        vectors_config=VectorParams(
            size=len(points[0]["vector"]),   # 384
            distance=Distance.COSINE         # Cosine similarity
        ),
        hnsw_config=HnswConfigDiff(
            m=16,                # HNSW: connections per node HNSW = Hierarchical Navigable Small World
            ef_construct=200,    # HNSW: search depth during build
            full_scan_threshold=10000  # Switch to brute-force if small
        ),
    )
    print(f"Collection '{COLLECTION_NAME}' created.")

    # ── 5. Upload in batches (faster, safer)
    BATCH = 100
    for i in range(0, len(points), BATCH):
        batch = points[i:i+BATCH]  # Slice: 100 items at a time

        # ── 6. Convert dict → Qdrant PointStruct
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=[
                PointStruct(
                    id=p["id"],
                    vector=p["vector"],
                    payload=p["payload"]
                ) for p in batch
            ]
        )
        print(f"  → Upserted batch {i//BATCH + 1}")

    # ── 7. Done!
    print("Vector DB built successfully!")


# ==============================================================
# MAIN: Run all steps
# ==============================================================
if __name__ == "__main__":
    print("=== BUILDING VECTOR DATABASE ===\n")
    docs = load_documents()
    chunks = chunk_documents(docs)
    points = embed_chunks(chunks)
    build_qdrant_index(points)
    Path("vector_db_built.flag").touch()
    print("\n" + "="*60)
    print("SUCCESS! Vector DB is ready!")
    print("Next: Run 'python support_chat.py' for the AI chat interface.")
    print("="*60)

=== BUILDING VECTOR DATABASE ===


STEP 0: INGEST RAW DOCUMENTS
Loading documents...
  → SupportCo Online Support Personnel Instruction Manual.pdf
Loaded 31 sections.

STEP 1: CHUNKING
Created 103 chunks.

STEP 2: EMBEDDING
Generating embeddings...
  → Embedded 100/103 chunks
Embedded 103 vectors.

STEP 3: INDEXING & STORAGE IN QDRANT
Deleted existing collection: support_docs
Collection 'support_docs' created.
  → Upserted batch 1
  → Upserted batch 2
Vector DB built successfully!

SUCCESS! Vector DB is ready!
Next: Run 'python support_chat.py' for the AI chat interface.


In [ ]:
# ==============================================================
# support_chat.py - AI SUPPORT CHAT GUI with OpenAI GPT
# Uses .env for API key | Modern OpenAI v1.0+ API
# Answers questions using your PDF + GPT-3.5/GPT-4
# Requires: vector DB built + Qdrant running
# ==============================================================

import tkinter as tk
from tkinter import scrolledtext, messagebox
from typing import List
import os
from pathlib import Path
from dotenv import load_dotenv  # ← Loads .env file

# ================================
# LOAD ENVIRONMENT VARIABLES
# ================================
load_dotenv()  # Reads .env file in project root

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError(
        "OPENAI_API_KEY not found!\n"
        "Create a .env file in your project folder with:\n"
        "OPENAI_API_KEY=sk-...\n"
        "Or run: export OPENAI_API_KEY='sk-...'"
    )

# ================================
# CONFIG
# ================================
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "support_docs"
TOP_K = 3
OPENAI_MODEL = "gpt-5-chat-latest"  # or gpt-3.5-turbo or "gpt-4o" for better quality

# ================================
# INITIALIZE CLIENTS
# ================================
print("\n" + "="*60)
print("INITIALIZING AI SUPPORT CHAT WITH OPENAI")
print("="*60)

from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from openai import OpenAI  # ← Modern OpenAI v1.0+

print("Loading embedding model...")
model = SentenceTransformer(EMBEDDING_MODEL)
client = QdrantClient(url=QDRANT_URL)
openai_client = OpenAI(api_key=OPENAI_API_KEY)  # ← Secure client


# ================================
# STEP 5: EMBED USER QUERY
# ================================
def embed_query(query: str):
    # ← Takes the user's question as a string (e.g., "How do I reset my password?")

    print("\n Embedding user query...")
    # ← Tells you in the console: "Now turning your question into AI understanding"

    query_vec = model.encode(query).tolist()
    # ← THE MAGIC HAPPENS HERE:
    #    • model = your local SentenceTransformer (all-MiniLM-L6-v2)
    #    • .encode() = converts text → 384-dimensional vector (meaning in numbers)
    #    • .tolist() = turns NumPy array → regular Python list (Qdrant needs this)

    print(f"  → Query vector generated (dim={len(query_vec)})")
    # ← Confirms success: "We now have a 384-number 'fingerprint' of your question"

    return query_vec
    # ← Returns the vector so Qdrant can search for similar chunks


# ================================
# STEP 6 and 7: SEARCH QDRANT; Retrieve and Rank
# ================================
def search_qdrant(query: str) -> List[str]:
    # Takes the user's question (string) and returns a list of text chunks

    print("Searching Qdrant vector database...")
    # Lets you know in the console: "Now looking for answers in your manual"

    query_vec = embed_query(query)
    # Turns your question into a 384D vector (meaning fingerprint)
    # So Qdrant can compare it to the chunks from your PDF

    search_result = client.search(
        collection_name=COLLECTION_NAME,   # "support_docs" — your knowledge base
        query_vector=query_vec,            # The vector of your question
        limit=TOP_K,                       # Only return top 3 matches (fast + accurate)
        with_payload=True,                 # Include the actual text and source
    )
    # This is the SEARCH — finds chunks most similar in meaning

    contexts = []  # Will hold the text chunks to show the user
    print(f"  → Found {len(search_result)} relevant chunks:")

    for i, hit in enumerate(search_result):
        # Loop through the top matches
        text = hit.payload.get("text", "")           # The actual text from the PDF
        source = hit.payload.get("source", "Unknown") # Which file it came from
        score = hit.score                            # How similar (0.0 to 1.0)

        print(f"     [{i+1}] Score: {score:.3f} | Source: {source}")
        # Shows you how good each match is — higher score = better match

        contexts.append(f"[From: {source}]\n{text}\n")
        # Saves the chunk + source in a nice format for the final answer

    return contexts
    # Returns the list of text chunks so GPT can turn them into a natural answer

# ================================
# STEP 8: GENERATE ANSWER WITH OPENAI (v1.0+)
# ================================
def generate_answer(query: str, contexts: List[str]) -> str:
    # Takes the user's question + retrieved chunks → returns a natural answer

    print("\nGenerating answer with OpenAI GPT...")
    # Tells you: "Now asking GPT to write a nice response"

    if not contexts:
        # No relevant info found in the manual
        return "I'm sorry, I couldn't find any information about that in the support manual."

    context_str = "\n\n".join(contexts)
    # Combines all retrieved chunks into one big string
    # Each chunk already has "[From: ...]" and the text

    system_prompt = f"""
You are a helpful, accurate, and friendly support assistant for Supportco.
Your knowledge comes ONLY from the Supportco Manual provided below.

RULES:
1. Answer the user's ORIGINAL QUESTION using ONLY the provided context.
2. Use the original question to guide your tone, focus, and relevance.
3. If the answer is not in the context, say: "I don't have that information in the manual."
4. Be concise, clear, and step-by-step.
5. Cite the source (e.g., "From: Supportco Manual.pdf") when possible.
6. Never guess or make up information.
7. Always use normal and polite English - don't cite technical terms or abbreviations - always translate abbreviations into plain English
8. Please don't ask users to perform additional steps unless you can describe the next steps from the provided context

CONTEXT FROM SUPPORTCO MANUAL:
{context_str.strip()}
"""
    # This is the AI's "brain" — its personality, rules, and knowledge
    # It knows: "Only answer from this text. Be nice. Be accurate."

    try:
        response = openai_client.chat.completions.create(
            model=OPENAI_MODEL,                    # e.g., "gpt-4o" or "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": system_prompt},  # ← AI's instructions + context
                {"role": "user",   "content": query}          # ← The actual user question
            ],
            temperature=0.3,   # Low = consistent, accurate answers (not creative)
            max_tokens=300     # Limit answer length
        )
        # Sends everything to OpenAI and gets back a natural response

        answer = response.choices[0].message.content.strip()
        # Extracts the actual text answer from OpenAI's response

        return answer
        # Returns the final, beautiful answer to show in the chat

    except openai.AuthenticationError:
        # Wrong or missing API key
        return "OpenAI API key is invalid. Check your .env file."

    except openai.RateLimitError:
        # Too many requests (free tier limit)
        return "OpenAI rate limit reached. Try again in a moment."

    except Exception as e:
        # Any other error (network, server, etc.)
        return f"Error with OpenAI: {str(e)}"


# ================================
# GUI: Support Chat Interface
# ================================
class SupportChatGUI:
    # This class builds and controls the entire chat window
    
    def __init__(self, root):
        # __init__ runs when you do SupportChatGUI(root)
        # root = the main Tkinter window (created with tk.Tk())
        
        self.root = root
        # Save the main window so we can use it later

        self.root.title("Supportco AI Support Assistant (OpenAI + RAG)")
        # Sets the title at the top of the window

        self.root.geometry("850x650")
        # Sets window size: 850px wide, 650px tall

        self.root.configure(bg="#f0f2f5")
        # Sets background color (light gray — clean look)

        # === HEADER TEXT ===
        header = tk.Label(
            root,
            text="Supportco AI Assistant",
            font=("Helvetica", 16, "bold"),
            bg="#f0f2f5",
            fg="#1a1a1a"
        )
        header.pack(pady=10)
        # Creates the big title at the top

        # === CHAT DISPLAY BOX ===
        self.chat_display = scrolledtext.ScrolledText(
            root,
            wrap=tk.WORD,           # Wrap text at word boundaries
            width=90,
            height=28,
            font=("Helvetica", 11),
            bg="white",
            fg="#1a1a1a",
            state=tk.DISABLED       # User can't type directly in it
        )
        self.chat_display.pack(padx=20, pady=10, fill=tk.BOTH, expand=True)
        # This is the main chat area where all messages appear

        # === INPUT AREA (bottom) ===
        input_frame = tk.Frame(root, bg="#f0f2f5")
        input_frame.pack(padx=20, pady=10, fill=tk.X)
        # A container to hold the text box + Send button

        self.entry = tk.Entry(
            input_frame,
            font=("Helvetica", 12),
            relief=tk.FLAT,
            bg="white",
            fg="#1a1a1a"
        )
        self.entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 10))
        # The box where the user types their question

        self.entry.bind("<Return>", self.send_message)
        # Pressing Enter = same as clicking Send

        send_btn = tk.Button(
            input_frame,
            text="Send",
            command=self.send_message,   # Click → run send_message()
            bg="#007bff",                # Blue background
            fg="white",
            font=("Helvetica", 11, "bold"),
            relief=tk.FLAT,
            cursor="hand2"               # Hand cursor on hover
        )
        send_btn.pack(side=tk.RIGHT)
        # The blue "Send" button

        # === WELCOME MESSAGE ===
        self.add_message("Assistant", "Hello! I'm your AI support assistant. Ask me anything about Supportco!")
        # Shows a friendly greeting when the app opens


    def add_message(self, sender: str, message: str):
        # Adds a new message to the chat display
        self.chat_display.config(state=tk.NORMAL)     # Unlock to edit
        self.chat_display.insert(tk.END, f"{sender}: {message}\n\n")
        self.chat_display.config(state=tk.DISABLED)   # Lock again
        self.chat_display.see(tk.END)                 # Auto-scroll to bottom


    def send_message(self, event=None):
        # Runs when user presses Enter or clicks Send
        
        query = self.entry.get().strip()
        # Get what the user typed and remove extra spaces
        
        if not query:
            return  # Do nothing if empty
        
        self.add_message("You", query)
        # Show the user's question in the chat
        
        self.entry.delete(0, tk.END)
        # Clear the input box
        
        self.add_message("Assistant", "Searching manual and generating answer...")
        # Show "thinking" message

        try:
            contexts = search_qdrant(query)           # Step 1: Search your manual
            answer = generate_answer(query, contexts) # Step 2: Ask GPT to write answer
            # Use after() so GUI stays responsive
            self.chat_display.after(100, lambda: self.add_message("Assistant", answer))
        except Exception as e:
            # If anything goes wrong (no internet, bad key, etc.)
            self.add_message("Assistant", f"Error: {e}")

# ================================
# LAUNCH GUI
# ================================
if __name__ == "__main__":
    # This line means: "Only run the code below if this file is executed directly"
    # (e.g., you type: python support_chat.py)
    # If someone imports this file, this part is skipped

    if not Path("vector_db_built.flag").exists():
        # Check: Does the file "vector_db_built.flag" exist in the folder?
        # This file is created by build_vector_db.py when it finishes successfully

        messagebox.showerror(
            "Vector DB Not Found",                                    # Title of pop-up
            "Please run build_vector_db.py first to create the vector database!"
        )
        # If the flag is missing → show a red error pop-up and stop
        # Prevents the app from starting without a knowledge base

    else:
        # The flag exists → everything is ready to go
        
        root = tk.Tk()
        # Create the main window (the "box" that holds your entire app)
        # root = tk.Tk() ← Step 1: Create the window (from Tkinter)

        app = SupportChatGUI(root)
        # Build your chat interface inside that window
        # app = SupportChatGUI(root) ← Step 2: Put your chat inside the window
        # This runs all the __init__ code: title, chat box, input field, etc.

        root.mainloop()
        # Starts the app
        # root.mainloop() ← Step 3: Turn on the app
        # mainloop() is inside the Tk class which is part of Tkinter which listens for mouse clicks, redraws the window etc.
        # Keeps the window open and responsive until you close it
        # This is the "on" switch for your entire AI assistant


INITIALIZING AI SUPPORT CHAT WITH OPENAI
Loading embedding model...
Searching Qdrant vector database...

 Embedding user query...
  → Query vector generated (dim=384)
  → Found 3 relevant chunks:
     [1] Score: 0.399 | Source: SupportCo Online Support Personnel Instruction Manual.pdf
     [2] Score: 0.381 | Source: SupportCo Online Support Personnel Instruction Manual.pdf
     [3] Score: 0.337 | Source: SupportCo Online Support Personnel Instruction Manual.pdf

Generating answer with OpenAI GPT...


/var/folders/zr/8xlf44gd1yng__g6g_09_xnr0000gn/T/ipykernel_72512/1716993564.py:90: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(
